<a href="https://colab.research.google.com/github/justinmolina/vhsupscale/blob/main/Video2X.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# [Video2X](https://github.com/k4yt3x/video2x) Colab Notebook

If you encounter any issues or have any questions or suggestions, post an issue [here](https://github.com/k4yt3x/video2x/issues/new).

Licensed under the ISC License: https://www.isc.org/licenses/. \
Copyright 2022-2024 K4YT3X and contributors.

Last updated on December 15, 2024 for version [6.2.0](https://github.com/k4yt3x/video2x/releases/tag/6.2.0).

In [ ]:
#@title ## Step 0: Runtime Selection

#@markdown **You'll need to select a GPU runtime for Video2X to work.** On the menu bar at the top, select:

#@markdown > Runtime > Change runtime type > Hardware accelerator

#@markdown Change "Hardware accelerator" to GPU and click "Save." After you're done, execute this cell to see the detailed information about the GPU allocated to you (optional).

!nvidia-smi

In [ ]:
#@title ## Step 1: Install Video2X and Dependencies

#@markdown **This step installs Video2X and its dependencies.**

!curl -LO https://github.com/k4yt3x/video2x/releases/download/6.2.0/video2x-linux-ubuntu2204-amd64.deb
!add-apt-repository -y ppa:ubuntuhandbook1/ffmpeg7
!apt-get update
!apt-get install -y libvulkan1 nvidia-driver-535 ffmpeg ./video2x-linux-ubuntu2204-amd64.deb

%env VK_ICD_FILENAMES=/usr/share/vulkan/icd.d/nvidia_icd.json

In [ ]:
#@title ## Step 2: Select/Upload Input File

#@markdown **Select or upload the file to process.**

#@markdown - ⚠️You might need to grant full access. Otherwise `drive.mount` may return an error.
#@markdown - Google Drive will be used for storage so it will be mounted in this step.
#@markdown - Select/upload a file after mounting Google Drive.

import pathlib

import ipywidgets as widgets
from google.colab import drive, files
from IPython.display import display

drive.mount("/content/drive", force_remount=True)
gdrive_path = pathlib.Path("/content/drive/MyDrive")

input_file = None

def on_dropdown_change(change):
    global input_file
    if change["name"] == "value":
        input_file = pathlib.Path(change["new"])
        print(f"Selected input file: {input_file}")

def upload_new_file(button):
    global input_file
    uploaded = files.upload()

    # Check if the user canceled the upload
    if not uploaded:
        print("Upload canceled.")
        return

    for file_name in uploaded.keys():
        input_file = pathlib.Path("/content") / file_name
        print(f"Uploaded and set input file: {input_file}")

file_dropdown = widgets.Dropdown(
    options=list(gdrive_path.iterdir()), description="Select file:"
)
file_dropdown.observe(on_dropdown_change)
display(file_dropdown)

upload_button = widgets.Button(description="Upload New File")
upload_button.on_click(upload_new_file)
display(upload_button)

In [ ]:
#@title ## Step 3A: Upscale

#@markdown **Fill in the form and run this cell to upscale a video.**

#@markdown - ⚠️ Remember free sessions can run for [at most 12 hours](https://research.google.com/colaboratory/faq.html#idle-timeouts). If you exceed this limit your progress could be lost. You can get [Colab Pro/Pro+](https://colab.research.google.com/signup/pricing) for longer runtimes.
#@markdown - It may be a good idea to upscale a small sample of your video to see how well you like the results before processing the whole video.

import pathlib
import subprocess

if 'input_file' not in globals() or input_file is None:
    raise NameError("Input file not specified. Did you forget to run the previous step?")

if 'gdrive_path' not in globals():
    raise NameError("Google Drive path not specified. Did you forget to run the previous step?")

#@markdown <br>

#@markdown ### Filter Select
#@markdown - ⚠️ `libplacebo` currently has compatibility issues with Colab's environment.
#@markdown - Both `realesrgan` and `libplacebo` can be used for anime/cartoon videos.
#@markdown - Choose `realesrgan` for real-life videos.

filter = "realesrgan" #@param ["realesrgan", "libplacebo"]

# Width and height of the output video. You can fill in one of them and the other one will be calculated automatically if it is left as `0`.
#@markdown <br>

#@markdown ### RealESRGAN Options
#@markdown - Choose `realesr-animevideov3` or `realesrgan-plus-anime` for anime/cartoon videos.
#@markdown - Choose `realesrgan-plus` for real-life videos.
#@markdown - `realesr-animevideov3` supports 2x, 3x, and 4x. Other models support only 4x.

model = "realesr-animevideov3" #@param ["realesrgan-plus-anime", "realesrgan-plus", "realesr-animevideov3"]
scale = 4 #@param {type:"slider", min:2, max:4, step:1}

#@markdown <br>

#@markdown ### libplacebo Options

#@markdown - `anime4k-v4-a`: Optimized for 1080p anime
#@markdown - `anime4k-v4-b`: Optimized for 720p anime
#@markdown - `anime4k-v4-c`: Optimized for 480p anime
#@markdown - `anime4k-v4-a+a`: Higher quality version of mode A
#@markdown - `anime4k-v4-b+b`: Higher quality version of mode B
#@markdown - `anime4k-v4-c+a`: Higher quality version of mode C
#@markdown - `anime4k-v4.1-gan`: Experimental SRGAN shaders; delivers the highest quality but is slower than standard modes

shader = "anime4k-v4-a" #@param ["anime4k-v4-a", "anime4k-v4-a+a", "anime4k-v4-b", "anime4k-v4-b+b", "anime4k-v4-c", "anime4k-v4-c+a", "anime4k-v4.1-gan"]
width = 3840 #@param {type:"number"}
height = 2160 #@param {type:"number"}

#@markdown <br>

#@markdown ### Advanced Options

#@markdown You typically don't need to touch these if you don't know what they do.

codec = "libx264" #@param {type:"string"}
preset = "slow" #@param {type:"string"}
pixfmt = "auto" #@param {type:"string"}
bitrate = 0 #@param {type:"number"}
crf = 20 #@param {type:"number"}
loglevel = "info" #@param ["trace", "debug", "info", "warn", "error", "critical", "none"]

#no_copy_streams = True # @param {type:"boolean"}

output_file = gdrive_path / f"{input_file.stem}.processed{input_file.suffix}"
if output_file.exists():
    raise FileExistsError(f"Cannot continue: file {output_file} already exists")
print(f"Output will be written to: {output_file}")

command = [
    "video2x",

    # Common options
    "--input",
    f"\"{input_file}\"",
    "--output",
    f"\"{output_file}\"",
    "--processor",
    filter,

    # Encoder options
    "--codec",
    codec,
    "--bit-rate",
    str(bitrate),
    "--log-level",
    loglevel,
    "-e",
    f"preset={preset}",
    "-e",
    f"crf={crf}"
]

# RealESRGAN options
if filter == "realesrgan":
    command.extend([
        "--realesrgan-model",
        model,
        "--scaling-factor",
        str(scale)
    ])

# libplacebo options
elif filter == "libplacebo":
    command.extend([
        "--width",
        str(width),
        "--height",
        str(height),
        "--libplacebo-shader",
        f"\"{shader}\""
    ])

if pixfmt != "auto":
    command.extend([
        "--pix-fmt",
        pixfmt
    ])

!{' '.join(command)}

In [ ]:
#@title ## Step 3B: Frame Interpolation 🚧 BROKEN 🚧

#@markdown **Fill in the form and run this cell to upscale a video.**

#@markdown - ⚠️ RIFE is currently broken on Colab for unknown reasons. SPIRV will fail to compile.
#@markdown - ⚠️ Remember free sessions can run for [at most 12 hours](https://research.google.com/colaboratory/faq.html#idle-timeouts). If you exceed this limit your progress could be lost. You can get [Colab Pro/Pro+](https://colab.research.google.com/signup/pricing) for longer runtimes.
#@markdown - It may be a good idea to upscale a small sample of your video to see how well you like the results before processing the whole video.

import pathlib
import subprocess

if 'input_file' not in globals() or input_file is None:
    raise NameError("Input file not specified. Did you forget to run the previous step?")

if 'gdrive_path' not in globals():
    raise NameError("Google Drive path not specified. Did you forget to run the previous step?")

#@markdown <br>

#@markdown ### Interpolator Select
#@markdown - Choose `rife` for real-life videos.

interpolator = "rife" #@param ["rife"]

# Width and height of the output video. You can fill in one of them and the other one will be calculated automatically if it is left as `0`.
#@markdown <br>

#@markdown ### Rife Options

model = "rife-v4.6" #@param ["rife", "rife-HD", "rife-UHD", "rife-anime", "rife-v2", "rife-v2.3", "rife-v2.4", "rife-v3.0", "rife-v3.1", "rife-v4", "rife-v4.6"]
frm_rate_mul = 2 #@param {type:"slider", min:2, max:10, step:1}

#@markdown <br>

#@markdown ### Advanced Options

#@markdown You typically don't need to touch these if you don't know what they do.

codec = "libx264" #@param {type:"string"}
preset = "slow" #@param {type:"string"}
pixfmt = "auto" #@param {type:"string"}
bitrate = 0 #@param {type:"number"}
crf = 20 #@param {type:"number"}
loglevel = "info" #@param ["trace", "debug", "info", "warn", "error", "critical", "none"]

#no_copy_streams = True # @param {type:"boolean"}

output_file = gdrive_path / f"{input_file.stem}.processed{input_file.suffix}"
if output_file.exists():
    raise FileExistsError(f"Cannot continue: file {output_file} already exists")
print(f"Output will be written to: {output_file}")

command = [
    "video2x",

    # Common options
    "--input",
    f"\"{input_file}\"",
    "--output",
    f"\"{output_file}\"",
    "--processor",
    interpolator,

    # Encoder options
    "--codec",
    codec,
    "--bit-rate",
    str(bitrate),
    "--log-level",
    loglevel,
    "-e",
    f"preset={preset}",
    "-e",
    f"crf={crf}"
]

# RIFE options
if interpolator == "rife":
    command.extend([
        "--rife-model",
        model,
        "--frame-rate-mul",
        str(frm_rate_mul)
    ])

if pixfmt != "auto":
    command.extend([
        "--pix-fmt",
        pixfmt
    ])

!{' '.join(command)}